In [7]:
import torch
from torch.utils.data import Dataset, DataLoader


# Custom Dataset class for .pt files
class TensorDataset(Dataset):
    def __init__(self, data_path):
        # Load preprocessed data (assumes the .pt file contains a tuple: (features, labels))
        data = torch.load(data_path)
        self.features = data[0]
        self.labels = data[1]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# Load train and test datasets
train_dataset = TensorDataset("data/processed/train.pt")
test_dataset = TensorDataset("data/processed/test.pt")

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\vasco\AppData\Local\Temp\ipykernel_12688\2120041566.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(data_path)


In [8]:
import torchvision.models as models
import torch.nn as nn

# Load MobileNetV3
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.mobilenet_v3_small(pretrained=True)

# Adjust the classifier to match the number of classes in your dataset
num_classes = train_dataset.labels.max().item() + 1  # Assuming labels start at 0
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model = model.to(device)


In [9]:
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimiser
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [2]:
import torch

torch.cuda.is_available()

False

In [11]:
# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels)

        # Backward pass and optimisation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

Epoch 1/50, Loss: 2.6227766861387756
Epoch 2/50, Loss: 2.419217560908584
Epoch 3/50, Loss: 2.228548150907985
Epoch 4/50, Loss: 2.014159303109597
Epoch 5/50, Loss: 1.8083262962986335
Epoch 6/50, Loss: 1.5997306678241459
Epoch 7/50, Loss: 1.4198890113696372
Epoch 8/50, Loss: 1.2490545576385887
Epoch 9/50, Loss: 1.1068845372821778
Epoch 10/50, Loss: 0.9767677386694956
Epoch 11/50, Loss: 0.9274435234841442
Epoch 12/50, Loss: 0.7815996323333001
Epoch 13/50, Loss: 0.7279738289134587
Epoch 14/50, Loss: 0.7239847597133151
Epoch 15/50, Loss: 0.8494234725022182
Epoch 16/50, Loss: 0.6128400540150576
Epoch 17/50, Loss: 0.53132944284304
Epoch 18/50, Loss: 0.701073273862616
Epoch 19/50, Loss: 0.5960126518434364
Epoch 20/50, Loss: 0.5160091888669731
Epoch 21/50, Loss: 0.48227172370084453
Epoch 22/50, Loss: 0.4771961641985413
Epoch 23/50, Loss: 0.4698681140189211
Epoch 24/50, Loss: 0.461262181516576
Epoch 25/50, Loss: 0.4377459495574739
Epoch 26/50, Loss: 0.47255786532784266
Epoch 27/50, Loss: 0.41818

In [14]:
import math

math.exp(-0.6902741293298817) * 100

50.143859119857645

In [12]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 4.91%


In [13]:
torch.save(model.state_dict(), "models/trained_lightweight_model.pth")